# PyTorchによる基本的な訓練の流れ

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/py-img-gen/python-image-generation/blob/main/notebooks/2-2_pytorch.ipynb)

## 準備

In [ ]:
!pip install -qq py-img-gen

In [ ]:
import pathlib

current_dir = pathlib.Path.cwd()
project_dir = current_dir / "data" / "pytorch-introduction"

project_dir.mkdir(parents=True, exist_ok=True)

## PyTorch の導入

- 参考: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

## データの扱い方

### データのダウンロードと読み込み

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

data_dir = project_dir / "dataset"

train_data = datasets.FashionMNIST(
    root=data_dir,
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root=data_dir,
    train=False,
    download=True,
    transform=ToTensor(),
)

### データローダーの作成

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64

train_dataloader = DataLoader(
    train_data, batch_size=batch_size
)
test_dataloader = DataLoader(
    test_data, batch_size=batch_size
)

### データローダーを用いたミニバッチの作成

In [ ]:
(Xs, ys) = next(iter(train_dataloader))
print(f"Shape of Xs [N, C, H, W]: {Xs.shape}")
print(f"Shape of ys: {ys.shape} {ys.dtype}")

### 訓練データの可視化

In [ ]:
from py_img_gen.datasets import get_fashion_mnist_classes

classes = get_fashion_mnist_classes()
print(classes)

In [ ]:
import matplotlib.pyplot as plt

idx = 0
# shape (X): (1, 28, 28) -> (28, 28)
X, y = Xs[idx].squeeze(dim=0), ys[idx]

fig, ax = plt.subplots()
ax.imshow(X, cmap="gray")
ax.set_title(f"Class: {y} ({classes[y]})")

## モデルの構築

### デバイスの取得

In [ ]:
import torch

device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"
)
print(f"Using {device} device")

## モデルの定義

In [ ]:
from torch import nn


class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)


model = NeuralNetwork()
model = model.to(device)

## モデルパラメータの最適化

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### モデルの訓練方法の定義

In [ ]:
def train(
    dataloader: DataLoader,
    model: NeuralNetwork,
    loss_fn: nn.CrossEntropyLoss,
    optimizer: torch.optim.SGD,
) -> None:
    model.train()  # モデルを訓練モードに

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 予測誤差の計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # 逆伝播の実行
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 != 0:
            continue

        current = (batch + 1) * len(X)
        size = len(dataloader.dataset)
        print(
            f"loss: {loss.item():>7f} "
            f"[{current:>5d}/{size:>5d}]"
        )

### モデルの評価方法の定義

In [ ]:
@torch.no_grad()
def test(
    dataloader: DataLoader,
    model: NeuralNetwork,
    loss_fn: nn.CrossEntropyLoss,
) -> None:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (
            (pred.argmax(1) == y)
            .type(torch.float)
            .sum()
            .item()
        )
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, "
        f"Avg loss: {test_loss:>8f} \n"
    )

### モデルの訓練と評価の実行

In [ ]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

## モデルの保存

In [ ]:
save_path = project_dir / "model.pth"
torch.save(model.state_dict(), save_path)
print(f"Saved PyTorch Model State to {save_path}")

## モデルの読み込み

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(
    state_dict=torch.load(save_path, weights_only=True)
)

## 学習済みモデルを用いた予測

### 予測の実行

In [ ]:
model.eval()

(x, y) = test_data[0]

with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = (
        classes[pred[0].argmax(0)],
        classes[y],
    )

print(f'Predicted: "{predicted}", Actual: "{actual}"')

### 予測結果の可視化

In [ ]:
x = x.cpu()
x = x.squeeze(dim=0)

fig, ax = plt.subplots()
ax.imshow(x, cmap="gray")
ax.set_title(f"Class: {y} ({classes[y]})")